# Домашнее задание номер 4 
# Макарова Мария

In [1]:
#Загрузка всего
import numpy as np
import cv2
import copy
ref_path = "../data/ref.jpg"
vid_path = "../data/vid.mp4"

reference = cv2.imread(ref_path)
cam = cv2.VideoCapture(vid_path)

In [2]:
#Читаем референс
pic = cv2.imread(ref_path) 

ORB: An efficient alternative to SIFT or SURF . It is a good alternative to SIFT and SURF in computation cost, matching performance and mainly the patents. 

ORB is basically a fusion of FAST keypoint detector and BRIEF descriptor with many modifications to enhance the performance. First it use FAST to find keypoints, then apply Harris corner measure to find top N points among them. It also use pyramid to produce multiscale-features. But one problem is that, FAST doesn't compute the orientation. So what about rotation invariance? Authors came up with following modification.

It computes the intensity weighted centroid of the patch with located corner at center. The direction of the vector from this corner point to centroid gives the orientation. To improve the rotation invariance, moments are computed with x and y which should be in a circular region of radius r, where r is the size of the patch.

Now for descriptors, ORB use BRIEF descriptors. But we have already seen that BRIEF performs poorly with rotation. So what ORB does is to "steer" BRIEF according to the orientation of keypoints. For any feature set of n binary tests at location (xi,yi), define a 2×n matrix, S which contains the coordinates of these pixels. Then using the orientation of patch, θ, its rotation matrix is found and rotates the S to get steered(rotated) version Sθ.

ORB discretize the angle to increments of 2π/30 (12 degrees), and construct a lookup table of precomputed BRIEF patterns. As long as the keypoint orientation θ is consistent across views, the correct set of points Sθ will be used to compute its descriptor.

BRIEF has an important property that each bit feature has a large variance and a mean near 0.5. But once it is oriented along keypoint direction, it loses this property and become more distributed. High variance makes a feature more discriminative, since it responds differentially to inputs. Another desirable property is to have the tests uncorrelated, since then each test will contribute to the result. To resolve all these, ORB runs a greedy search among all possible binary tests to find the ones that have both high variance and means close to 0.5, as well as being uncorrelated. The result is called rBRIEF.

For descriptor matching, multi-probe LSH which improves on the traditional LSH, is used. The paper says ORB is much faster than SURF and SIFT and ORB descriptor works better than SURF. 

In [3]:
orb = cv2.ORB_create()

In [4]:
#Сначала считаем дескрипторы и кей поинтс для референса
kp1, des1 = orb.detectAndCompute(pic, None)

In [5]:
#Посмотрим, что получилось
output_image = cv2.drawKeypoints(pic, kp1, 0, (0, 0, 255),flags=cv2.DRAW_MATCHES_FLAGS_NOT_DRAW_SINGLE_POINTS)

In [7]:
cv2.namedWindow("matchd", cv2.WINDOW_NORMAL)
cv2.resizeWindow("matchd", 600, 600) 
cv2.imshow("matchd",output_image)
cv2.waitKey(0)

-1

Brute-Force matcher is simple. It takes the descriptor of one feature in first set and is matched with all other features in second set using some distance calculation. And the closest one is returned.

Next we create a BFMatcher object with distance measurement cv.NORM_HAMMING (since we are using ORB) and crossCheck is switched on for better results. Then we use Matcher.match() method to get the best matches in two images. We sort them in ascending order of their distances so that best matches (with low distance) come to front.

In [8]:
bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)

In [ ]:
#Теперь пробуем на видео
while(True):
    ret, frame = cam.read()
    
    if (ret == False):
        cam.release()
        cam = cv2.VideoCapture(vid_path)
        continue
    
    
    kp2, des2 = orb.detectAndCompute(frame, None)
    matches = bf.match(des1, des2)
    matches = sorted(matches, key = lambda x:x.distance)
    
   # output_image = cv2.drawKeypoints(frame, kp2, 0, (0, 0, 255),
                               #  flags=cv2.DRAW_MATCHES_FLAGS_NOT_DRAW_SINGLE_POINTS)
   # cv2.imshow("matchd",output_image)
   # cv2.waitKey(0)
    
    result = cv2.drawMatches(pic, kp1, frame, kp2, matches[:30], None,flags=cv2.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS)
    cv2.namedWindow("matchd", cv2.WINDOW_NORMAL)
    cv2.resizeWindow("matchd", 600, 600) 
    cv2.imshow("matchd", result)
    
    key = cv2.waitKey(240) & 0xFF
        
    if (key == ord('q')):
        break

cam.release()
cv2.destroyAllWindows()
cv2.waitKey(10)

# Выводы : 
Я реализовала распознавание ключевых точек и дескрипторов на видео и сопоставление им дескрипторов референсной картинки. В самом начале видео распознавание отличное , однако , ближе к концу, когда в кадре появляются стул и удлинитель (причем таких же цветов как референсная деталька) и деталька проносится на их фоне, то алгоритм начинает ошибаться и сопоставлять именно их элементы . 

Ключевые точки не начинают ближе к концу рисоваться на детальке, потому что по мнению алгоритма она не вносит полезной информации , тк сливается с фоном стула и теряется на нем.

Удлинитель и стул вносят вклад в информацию о кадре , тк являются объектами на контрастном сером фоне --> их ключевые точки берутся в расчет алгоритмом.

Тк элементы удлинителя и стула схожи с элементами референса (особенно удлинителя ,  он горизонтальный , а мы на референсе в основном горизонтальные элементы натыкали ключевыми точками, как видно из 'output_image') , то мы и получаем ложное распознавание. Еще и кольцо пропадает из кадра ближе к концу видео , на него в первой половине видео мы в том числе ориентировались